In [1]:
import pandas as pd

In [2]:
connection_start = pd.read_csv("connection_starts.csv", delimiter='\t', names=['RT', 'SI', 'DI', 'SP', 'DP'])
connection_end = pd.read_csv("connection_ends.csv", delimiter='\t', names=['RT', 'SI', 'DI', 'SP', 'DP'])

In [3]:
connection_start.head(3)

,RT,SI,DI,SP,DP
0,117.346401,192.168.56.102,192.168.56.104,37426,8080
1,138.481849,192.168.56.102,192.168.56.104,1463,8080
2,138.482122,192.168.56.102,192.168.56.104,1464,8080


In [4]:
connection_end.head(3)

,RT,SI,DI,SP,DP
0,138.483416,192.168.56.104,192.168.56.102,8080,1463
1,138.483417,192.168.56.104,192.168.56.102,8080,1464
2,138.483418,192.168.56.104,192.168.56.102,8080,1465


In [5]:
connection_start['duration'] = 0

In [ ]:
processed_rows = []


for row in range(len(connection_start)):

    if row in processed_rows:
        continue

    SI = connection_start['SI'][row]
    DI = connection_start['DI'][row]
    SP = connection_start['SP'][row]
    DP = connection_start['DP'][row]
    
    mask_end = ((connection_end['SI'] == DI) & 
            (connection_end['DI'] == SI) & 
            (connection_end['SP'] == DP) & 
            (connection_end['DP'] == SP))
    
    mask_start = ((connection_start['SI'] == SI) & 
            (connection_start['DI'] == DI) & 
            (connection_start['SP'] == SP) & 
            (connection_start['DP'] == DP))
    
    matching_rows_end = connection_end[mask_end]
    matching_rows_start = connection_start[mask_start]

    matching_indices = connection_start[mask_start].index.tolist()
    processed_rows.extend(matching_indices)

    start_rt_values = matching_rows_start['RT'].values if not matching_rows_start.empty else []
    end_rt_values = matching_rows_end['RT'].values if not matching_rows_end.empty else []
    
    if not matching_rows_end.empty:

        len_start = len(start_rt_values)
        len_end = len(end_rt_values)
        
        durations = []
        
        min_len = min(len_start, len_end)
        for i in range(min_len):
            if i < len_end:
                duration = end_rt_values[i] - start_rt_values[i]
                durations.append(duration)
        
        max_len = max(len_start, len_end)
        for i in range(min_len, max_len):
            durations.append(100)
        
        if durations:
            connection_start.loc[mask_start, 'duration'] = durations
        else:
            connection_start.loc[mask_start, 'duration'] = [100]
    else:
        connection_start.loc[mask_start, 'duration'] = 100

/var/folders/jv/6j759gh52pjg5xgj136xc96w0000gn/T/ipykernel_49510/4191302886.py:51: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[np.float64(0.001566999999994323), np.float64(7.052814000000012), np.float64(21.93356799999998), np.float64(25.715428000000003), np.float64(59.192909000000014), np.float64(68.60786000000002), 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  connection_start.loc[mask_start, 'duration'] = durations


In [ ]:
connection_start.to_csv("duration.csv")